In [ ]:
!pip install tifffile pandas scikit-learn matplotlib geopandas rasterio albumentations segmentation_models_pytorch#opencv-python 

In [ ]:
%load_ext autoreload
%autoreload 2

import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""
import tifffile
import time
from datetime import datetime
import pandas as pd
pd.set_option('display.max_colwidth', 250)
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import geopandas as gpd
from shapely.geometry import Polygon
from shapely.geometry import shape
import glob
from shapely.geometry import box
import rasterio
from rasterio.windows import Window
import json
from matplotlib.colors import LinearSegmentedColormap
red_to_green = LinearSegmentedColormap.from_list("RedGreen", ["red", "green"])

from src.visualize import visualize_s2_concat_bands_path, visualize_s1_path, scale_img, visualize_ls_concat_bands_path
from src.visualize import visualize_cb_mux_path, visualize_cb_wpm_path
# from src.visualize import visualize_s2_concat_bands_path_old

In [ ]:
# import shutil
# from pathlib import Path

# for src_folder in glob.glob("1*"):
#     event_id = src_folder.split("/")[-1]
#     dst_folder = Path(f"data/{event_id}")  # will be created if it doesn't exist

#     # copies entire directory tree; preserves metadata (copy2)
#     shutil.copytree(
#         src_folder,
#         dst_folder,
#         dirs_exist_ok=True,                     # requires Python 3.8+
#         copy_function=shutil.copy2,             # preserve mtime/permissions
#         symlinks=False,                         # set True to keep symlinks as symlinks
#         ignore=shutil.ignore_patterns('__pycache__', '*.pyc', '.DS_Store')
#     )

# for src_folder in glob.glob("1*"):
#     shutil.rmtree(src_folder)

# for path in glob.glob("README_*"):
#     shutil.move(path, f"data/{path}")
# shutil.move("DATASET_OVERVIEW.md", f"data/DATASET_OVERVIEW.md")
# shutil.move("dataset_analysis.json", "data/dataset_analysis.json")
# shutil.move("mapbiomas_alerts.geojson", "data/mapbiomas_alerts.geojson")

In [ ]:
with open("data/dataset_analysis.json", "r", encoding="utf-8") as file:
    dataset_analysis = json.load(file)

In [ ]:
df = gpd.read_file("data/mapbiomas_alerts.geojson")
events_sorted_by_area = df.sort_values("areaHa", ascending=False)["alertCode"].tolist()
df.shape

In [ ]:
# json.loads(open("data/1389384/sentinel2/1389384_sentinel2_S2-16D_V2_018015_20250218_20250218_bands.json").read())

# alertCode = "1389384"
# with rasterio.open(f"data/1389384/sentinel2/1389384_sentinel2_S2-16D_V2_018015_20250218_20250218.tif") as src:
#     data = src.read()  # Shape: (bands, height, width)
# data.shape

# event_ids = [k.split("/")[-1] for k in sorted(glob.glob("data/1*"))]
# len(event_ids)#, event_ids

# nothing_count = 0
# for event_id in events_sorted_by_area:
# # for event_id in [k.split("/")[-1] for k in glob.glob(f"data/*")]:
#     if len(glob.glob(f"data/{event_id}/*/*.tif")) == 0:
#         print(f"data/{event_id}/*/*.tif")
#         nothing_count += 1
#     if len(glob.glob(f"data/{event_id}/*/*.tif")) < 10:
#         print(glob.glob(f"data/{event_id}/*/*.tif"))
#     # else:
#     #     print(glob.glob(f"data/{event_id}/*/*.tif"))
# nothing_count

In [ ]:
for event_id in events_sorted_by_area[2:]:
    s1_paths = sorted([k for k in glob.glob(f"data/{event_id}/sentinel1/*.tif") if not "_nrpb" in k and not "_rfdi" in k and not "_rvi" in k and not "_vv_vh_ratio" in k and not "uncompressed" in k])
    break

In [ ]:
s1_paths

In [ ]:
img.min(), img.max()

In [ ]:
path = 'data/1389572/sentinel1/1389572_sentinel1_S1A_IW_GRDH_1SDV_20250509T090650_20250509T090715_059115_0755A7_20250509.tif'
img = rasterio.open(path.replace(".tif", "_rvi.tif")).read(1)
plt.imshow(img, vmin=10000, vmax=20000)
plt.show()

In [ ]:
img.min(), img.max()

In [ ]:
img = rasterio.open(path.replace(".tif", "_rfdi.tif")).read(1)
plt.imshow(img, vmin=0, vmax=0.5)
plt.show()

In [ ]:
img = rasterio.open('data/1389384/sentinel1/1389384_sentinel1_S1A_IW_GRDH_1SDV_20250406T093308_20250406T093333_058634_07424A_20250406_nrpb.tif').read(1)
plt.imshow(img)
plt.show()

In [ ]:
def is_ax_empty(ax):
    """Check if a matplotlib axis is empty."""
    return not (ax.lines or ax.collections or ax.patches or ax.images)

for event_id in events_sorted_by_area[:]:
    s2_paths = sorted([k for k in glob.glob(f"data/{event_id}/sentinel2/*L2A*.tif") if not "ndvi" in k and not "ndwi" in k and not "uncompressed" in k])
    s2_16d_paths = sorted([k for k in glob.glob(f"data/{event_id}/sentinel2/*16D*.tif") if not "ndvi" in k and not "ndwi" in k and not "uncompressed" in k])
    s1_paths = sorted([k for k in glob.glob(f"data/{event_id}/sentinel1/*.tif") if not "_nrpb" in k and not "_rfdi" in k and not "_rvi" in k and not "_vv_vh_ratio" in k and not "uncompressed" in k])
    ls_paths = sorted([k for k in glob.glob(f"data/{event_id}/landsat/*_L2SP_*.tif") if not "_nbr" in k and not "_ndwi" in k and not "_ndvi" in k])
    cb_wpm_paths = sorted([k for k in glob.glob(f"data/{event_id}/cbers4a/*_WPM_*.tif") if not "_ndvi" in k])
    cb_mux_paths = sorted([k for k in glob.glob(f"data/{event_id}/cbers4a/*_MUX_*.tif") if not "_ndvi" in k])
    print(f"{event_id}: {len(s2_paths):2} S2 L2A, {len(s2_16d_paths):2} S2 16D,{len(s1_paths):2} S1, {len(ls_paths):2} LS L2SP, {len(cb_wpm_paths):2} CB WPM, {len(cb_mux_paths):2} CB MUX")
    paths = s2_paths + s1_paths + ls_paths + cb_wpm_paths + cb_mux_paths + s2_16d_paths
    if len(paths) == 0:
        continue
    mask_path = glob.glob(f"data/{event_id}/*_mask.tif")[0]
    
    # Sort by the date right before the .tif
    paths = sorted(paths, key=lambda p: datetime.strptime(p.split('_')[-1].split('.')[0], "%Y%m%d"))
    imgs_to_show, cloud_masks_to_show = [], []
    titles = []
    for path in paths:
        img_orig = tifffile.imread(path)
        date = path.split("/")[-1].split('.')[0].split('_')[-1]
        date_str = f"{date[:4]}-{date[4:6]}-{date[6:8]}"
        if "L2A" in path:
            scl = img_orig[:, :, -2].copy()
            img_orig = img_orig[:, :, [11, 8, 4]]
            img_orig = img_orig.astype(np.float32)
            if scl.max() > 11:
                if scl.max() > 50:
                    factor = 6
                elif scl.max() > 45:
                    factor = 5
                elif scl.max() >= 40:
                    factor = 4.5
                else:
                    factor = 4
                img_orig = (img_orig / factor).round(0)
                scl_clean = (scl / factor).round(0).astype('uint8')   # back to 0..11
            else:
                factor = 1
                scl_clean = scl.copy()
        
            img_orig[img_orig == 0] = np.nan

            # low values, but all minimum above 1000 ==> Subtract by 1000  (undo the PB 04.X radiometric offset)
            if (np.nanmin(img_orig, axis=(0,1)) > 1000).sum() == 3:
                img_orig = img_orig - 1000
                img_orig[img_orig < 0] = np.nan
            
            # if scl.max() > 11:
            #     f, ax = plt.subplots(1, 3, figsize=(25, 8))
            #     img = visualize_s2_concat_bands_path(path)
            #     ax[0].imshow(img)
            #     ax[1].imshow(scl, vmin=0, vmax=11)
            #     ax[1].set_title(f"Tampered values: {np.unique(scl)}", fontsize=16)
            #     ax[2].imshow(scl_clean, vmin=0, vmax=11)
            #     ax[2].set_title(f"Dividing by {factor}: {np.unique(scl_clean)}", fontsize=16)
            #     plt.tight_layout()
            #     plt.show()

            min_ = np.nanmin(img_orig, axis=(0,1)).round(0)
            max_ = np.nanmax(img_orig, axis=(0,1)).round(0)
            mean_ = np.nanmean(img_orig, axis=(0,1)).round(0)
            
            cloudmask = np.isin(scl_clean, [8, 9, 10]) # clouds only

            clouds_perc = 100 * cloudmask.sum() / (img_orig.shape[0] * img_orig.shape[1])
            nodata_perc = 100 * (img_orig[:, :, 2] == 0).sum() / (img_orig.shape[0] * img_orig.shape[1])
            s2_sat = path.split("/")[-1].split("_")[2]
            print(f"S2 {s2_sat} {date_str}: {clouds_perc:5.1f}% Clouds, {nodata_perc:5.1f}% Nodata")
            img = visualize_s2_concat_bands_path(path)
            titles.append(f"S2 {s2_sat}: {date_str} - factor {factor}\n{clouds_perc:.1f}% Clouds, {nodata_perc:.1f}% Nodata\nMin: {min_}, Max: {max_}\nMean: {mean_}")
        elif "16D" in path:
            scl = img_orig[:, :, -1].copy()
            img_orig = img_orig[:, :, [10, 7, 3]]
            img_orig = img_orig.astype(np.float32)
            if scl.max() > 11:
                if scl.max() > 50:
                    factor = 6
                elif scl.max() > 45:
                    factor = 5
                elif scl.max() >= 40:
                    factor = 4.5
                else:
                    factor = 4
                img_orig = (img_orig / factor).round(0)
                scl_clean = (scl / factor).round(0).astype('uint8')   # back to 0..11
            else:
                factor = 1
                scl_clean = scl.copy()
                
            img_orig[img_orig == 0] = np.nan
            # low values, but all minimum above 1000 ==> Subtract by 1000 (undo the PB 04.X radiometric offset)
            if (np.nanmin(img_orig, axis=(0,1)) > 1000).sum() == 3:
                img_orig = img_orig - 1000
                img_orig[img_orig < 0] = np.nan
            min_ = np.nanmin(img_orig, axis=(0,1)).round(0)
            max_ = np.nanmax(img_orig, axis=(0,1)).round(0)
            mean_ = np.nanmean(img_orig, axis=(0,1)).round(0)
            
            # if scl.max() > 11:
            #     f, ax = plt.subplots(1, 3, figsize=(25, 8))
            #     img = visualize_s2_concat_bands_path(path)
            #     ax[0].imshow(img)
            #     ax[1].imshow(scl, vmin=0, vmax=11)
            #     ax[1].set_title(f"Tampered values: {np.unique(scl)}", fontsize=16)
            #     ax[2].imshow(scl_clean, vmin=0, vmax=11)
            #     ax[2].set_title(f"Dividing by {factor}: {np.unique(scl_clean)}", fontsize=16)
            #     plt.tight_layout()
            #     plt.show()
            cloudmask = np.isin(scl_clean, [8, 9, 10]) # clouds only
            clouds_perc = 100 * cloudmask.sum() / (img_orig.shape[0] * img_orig.shape[1])
            nodata_perc = 100 * (img_orig[:, :, 1] == 0).sum() / (img_orig.shape[0] * img_orig.shape[1])
            print(f"S2 16D {date_str}: {clouds_perc:5.1f}% Clouds, {nodata_perc:5.1f}% Nodata")
            img = visualize_s2_concat_bands_path(path)
            titles.append(f"S2 16D: {date_str} - factor {factor}\n{clouds_perc:.1f}% Clouds, {nodata_perc:.1f}% Nodata\nMin: {min_}, Max: {max_}\nMean: {mean_}")
        elif "_L2SP" in path:
            min_ = img_orig[:, :, [7, 3, 0]].min(axis=(0,1)).round(0)
            max_ = img_orig[:, :, [7, 3, 0]].max(axis=(0,1)).round(0)
            mean_ = img_orig[:, :, [7, 3, 0]].mean(axis=(0,1)).round(0)
            
            qa = img_orig[:, :, -5].copy()
            # bit positions for dilated, cirrus, cloud
            CLOUD_BITS = (1, 2, 3)
            cloudmask = np.zeros_like(qa, dtype=bool)
            for b in CLOUD_BITS:
                cloudmask |= np.bitwise_and(qa, 1 << b) != 0
            cloudmask = cloudmask.astype(np.uint8)

            clouds_perc = 100 * (cloudmask.sum() / (img_orig.shape[0] * img_orig.shape[1]))
            nodata_perc = 100 * (img_orig[:, :, 2] == 0).sum() / (img_orig.shape[0] * img_orig.shape[1])
            print(f"LS     {date_str}: {clouds_perc:5.1f}% Clouds, {nodata_perc:5.1f}% Nodata")
            img = visualize_ls_concat_bands_path(path)
            titles.append(f"LS: {date_str}\n{clouds_perc:.1f}% Clouds, {nodata_perc:.1f}% Nodata\nMin: {min_}, Max: {max_}\nMean: {mean_}")
        elif "_WPM_" in path:
            min_ = img_orig.min(axis=(0,1)).round(0)
            max_ = img_orig.max(axis=(0,1)).round(0)
            mean_ = img_orig.mean(axis=(0,1)).round(0)

            cloudmask = (img_orig[:, :, 2] > 35000).astype(np.uint8)
            clouds_perc = 100 * cloudmask.sum() / (img_orig.shape[0] * img_orig.shape[1])
            nodata_perc = 100 * (img_orig[:, :, 0] == 0).sum() / (img_orig.shape[0] * img_orig.shape[1])
            print(f"WPM    {date_str}: {clouds_perc:5.1f}% Clouds, {nodata_perc:5.1f}% Nodata")
            img = visualize_cb_wpm_path(path)
            titles.append(f"WPM: {date_str}\n{clouds_perc:.1f}% Clouds, {nodata_perc:.1f}% Nodata\nMin: {min_}, Max: {max_}\nMean: {mean_}")
        elif "_MUX_" in path:
            min_ = img_orig[:, :, [3, 2, 1]].min(axis=(0,1)).round(0)
            max_ = img_orig[:, :, [3, 2, 1]].max(axis=(0,1)).round(0)
            mean_ = img_orig[:, :, [3, 2, 1]].mean(axis=(0,1)).round(0)

            cloudmask = (img_orig[:, :, 0] > 60).astype(np.uint8)
            clouds_perc = 100 * cloudmask.sum() / (img_orig.shape[0] * img_orig.shape[1])
            nodata_perc = 100 * (img_orig[:, :, 0] == 0).sum() / (img_orig.shape[0] * img_orig.shape[1])
            print(f"MUX    {date_str}: {clouds_perc:5.1f}% Clouds, {nodata_perc:5.1f}% Nodata")
            img = visualize_cb_mux_path(path)
            titles.append(f"MUX: {date_str}\n{clouds_perc:.1f}% Clouds, {nodata_perc:.1f}% Nodata\nMin: {min_}, Max: {max_}\nMean: {mean_}")
        else:
            min_ = img_orig.min(axis=(0,1)).round(0)
            max_ = img_orig.max(axis=(0,1)).round(0)
            mean_ = img_orig.mean(axis=(0,1)).round(0)
            cloudmask = np.zeros_like(img_orig[:, :, 0], dtype=np.uint8)
            nodata_perc = 100 * (img_orig[:, :, 0] == 0).sum() / (img_orig.shape[0] * img_orig.shape[1])
            print(f"S1     {date_str}: {nodata_perc:5.1f}% Nodata")
            img = visualize_s1_path(path)
            titles.append(f"S1: {date_str}\n{nodata_perc:.1f}% Nodata\nMin: {min_}, Max: {max_}\nMean: {mean_}")
        imgs_to_show.append(img)
        cloud_masks_to_show.append(cloudmask)

    if len(imgs_to_show) > 0:
        rows = (len(imgs_to_show) // 5) + 1
        cols =  (len(imgs_to_show) // rows) + 1
        f, ax = plt.subplots(rows, max(2, cols), figsize=(30, 7 * rows))
        ax = ax.flatten()
        for j in range(len(imgs_to_show)):
            ax[j].imshow(imgs_to_show[j])
            ax[j].imshow(np.ma.masked_where(cloud_masks_to_show[j] != 1, cloud_masks_to_show[j]), cmap="spring", alpha=0.5)
            ax[j].set_title(titles[j], fontsize=15)

        mask = rasterio.open(mask_path).read(1)
        date = mask_path.split("/")[-1].split('_')[1]
        date_str = f"{date[:4]}-{date[4:6]}-{date[6:8]}"
        if mask.shape[0] != img.shape[1]:
            mask = cv2.resize(mask, (img.shape[1], img.shape[0]), interpolation=cv2.INTER_NEAREST)
        ax[j+1].imshow(img)
        ax[j+1].imshow(np.ma.masked_where(mask != 255, mask), cmap="cool" if "_MUX_" in path else "autumn", alpha=0.5)
        ax[j+1].set_title(f"Deforestation date: {date_str}", fontsize=15)
        # Hide empty axes
        for ax_idx in range(len(ax)):
            if is_ax_empty(ax[ax_idx]):
                ax[ax_idx].set_visible(False)
        plt.tight_layout()
        plt.show()

In [ ]:
# # S2 16D IS SUPER MESSY, DONT USE
# def is_ax_empty(ax):
#     """Check if a matplotlib axis is empty."""
#     return not (ax.lines or ax.collections or ax.patches or ax.images)

# max_cloud_nodata = 100
# for event_id in events_sorted_by_area[:]:
#     # rand_int = np.random.randint(len(events_sorted_by_area))
#     # rand_int = k
#     # event_id = events_sorted_by_area[rand_int]
#     s2_paths = sorted([k for k in glob.glob(f"data/{event_id}/sentinel2/*L2A*_ndvi.tif")])
#     s2_16d_paths = sorted([k for k in glob.glob(f"data/{event_id}/sentinel2/*16D*_ndvi.tif")])
#     s1_paths = sorted([k for k in glob.glob(f"data/{event_id}/sentinel1/*.tif") if not "_nrpb" in k and not "_rfdi" in k and not "_rvi" in k and not "_vv_vh_ratio" in k])
#     ls_paths = sorted([k for k in glob.glob(f"data/{event_id}/landsat/*_L2SP_*ndvi.tif")])
#     cb_wpm_paths = sorted([k for k in glob.glob(f"data/{event_id}/cbers4a/*_WPM_*_ndvi.tif")])
#     cb_mux_paths = sorted([k for k in glob.glob(f"data/{event_id}/cbers4a/*_MUX_*_ndvi.tif")])
#     print(f"{event_id}: {len(s2_paths)} S2 L2A, {len(s1_paths)} S1, {len(ls_paths)} LS L2SP, {len(cb_wpm_paths)} CB WPM, {len(cb_mux_paths)} CB MUX")
#     paths = s2_paths + s1_paths + ls_paths + cb_wpm_paths + cb_mux_paths + s2_16d_paths
#     if len(paths) == 0:
#         continue
#     mask_path = glob.glob(f"data/{event_id}/*_mask.tif")[0]
    
#     # Sort by the date right before the .tif
#     paths = sorted(paths, key=lambda p: datetime.strptime(p.replace("_ndvi", "").split('_')[-1].split('.')[0], "%Y%m%d"))
#     imgs_to_show = []
#     titles = []
#     for path in paths:
#         img_orig = rasterio.open(path).read()
#         date = path.replace("_ndvi", "").split("/")[-1].split('.')[0].split('_')[-1]
#         date_str = f"{date[:4]}-{date[4:6]}-{date[6:8]}"
#         if "L2A" in path:
#             img_orig = img_orig[0]
#             # print(img_orig[:, :, 1:-2].min(axis=(0, 1)), img_orig[:, :, 1:-2].max(axis=(0, 1)))
#             nodata_perc = 100 * (img_orig == 0).sum() / (img_orig.shape[0] * img_orig.shape[1])
#             what_to_do_string = "SKIP"
#             if nodata_perc < max_cloud_nodata:
#                 what_to_do_string = "NO SKIP"
#             print(f"S2 L2A {date_str}: {nodata_perc:5.1f}% Nodata --> {what_to_do_string}")
#             if nodata_perc > max_cloud_nodata:
#                 continue
#             # img = visualize_s2_concat_bands_path(path)
#             ending = path.split("/")[-1].split('.')[0].split('_')[-1]
#             titles.append(f"S2 L2A: {date_str}\n{nodata_perc:.1f}% Nodata")
#         elif "16D" in path:
#             img_orig = img_orig[0]
#             # clouds_perc = 100 * (img_orig[:, :, 1] > 2400).sum() / (img_orig.shape[0] * img_orig.shape[1])
#             nodata_perc = 100 * (img_orig == 0).sum() / (img_orig.shape[0] * img_orig.shape[1])
#             what_to_do_string = "SKIP"
#             if nodata_perc < max_cloud_nodata:
#                 what_to_do_string = "NO SKIP"
#             print(f"S2 16D {date_str}: {nodata_perc:5.1f}% Nodata --> {what_to_do_string}")
#             if nodata_perc > max_cloud_nodata:
#                 continue
#             # img = visualize_s2_concat_bands_path(path)
#             ending = path.split("/")[-1].split('.')[0].split('_')[-1]
#             titles.append(f"S2 16D: {date_str}\n{nodata_perc:.1f}% Nodata")
#         elif "_L2SP" in path:
#             img_orig = img_orig[0]
#             nodata_perc = 100 * (img_orig == 0).sum() / (img_orig.shape[0] * img_orig.shape[1])
#             what_to_do_string = "SKIP"
#             if clouds_perc + nodata_perc < max_cloud_nodata:
#                 what_to_do_string = "NO SKIP"
#             print(f"LS     {date_str}: {nodata_perc:5.1f}% Nodata --> {what_to_do_string}")
#             if nodata_perc > max_cloud_nodata:
#                 continue
#             # img = visualize_ls_concat_bands_path(path)
#             ending = path.split("/")[-1].split('.')[0].split('_')[-1]
#             titles.append(f"LS: {date_str}\n{nodata_perc:.1f}% Nodata")
#         elif "_WPM_" in path:
#             img_orig = img_orig[0]
#             # clouds_perc = 100 * (img_orig[:, :, 2] > 35000).sum() / (img_orig.shape[0] * img_orig.shape[1])
#             nodata_perc = 100 * (img_orig == 0).sum() / (img_orig.shape[0] * img_orig.shape[1])
#             what_to_do_string = "SKIP"
#             if nodata_perc < max_cloud_nodata:
#                 what_to_do_string = "NO SKIP"
#             print(f"WPM    {date_str}: {nodata_perc:5.1f}% Nodata --> {what_to_do_string}")
#             if nodata_perc > max_cloud_nodata:
#                 continue
#             # img = visualize_cb_wpm_path(path)
#             ending = path.split("/")[-1].split('.')[0].split('_')[-1]
#             titles.append(f"WPM: {date_str}\n{nodata_perc:.1f}% Nodata")
#         elif "_MUX_" in path:
#             img_orig = img_orig[0]
#             # clouds_perc = 100 * (img_orig[:, :, 0] > 55).sum() / (img_orig.shape[0] * img_orig.shape[1])
#             nodata_perc = 100 * (img_orig == 0).sum() / (img_orig.shape[0] * img_orig.shape[1])
#             what_to_do_string = "SKIP"
#             if nodata_perc < max_cloud_nodata:
#                 what_to_do_string = "NO SKIP"
#             print(f"MUX    {date_str}: {nodata_perc:5.1f}% Nodata --> {what_to_do_string}")
#             if nodata_perc > max_cloud_nodata:
#                 continue
#             # img = visualize_cb_mux_path(path)
#             ending = path.split("/")[-1].split('.')[0].split('_')[-1]
#             titles.append(f"MUX: {date_str}\n{nodata_perc:.1f}% Nodata")
#         else:
#             if img_orig.shape[0] < 15:
#                 img_orig = np.transpose(img_orig, (1, 2, 0))
#             nodata_perc = 100 * (img_orig[:, :, 0] == 0).sum() / (img_orig.shape[0] * img_orig.shape[1])
#             what_to_do_string = "SKIP"
#             if nodata_perc < max_cloud_nodata:
#                 what_to_do_string = "NO SKIP"
#             date = path.split("/")[-1].split('.')[0].split('_')[-1]
#             date_str = f"{date[:4]}-{date[4:6]}-{date[6:8]}"
#             print(f"S1     {date_str}: {nodata_perc:5.1f}% Nodata --> {what_to_do_string}")
#             if nodata_perc > max_cloud_nodata:
#                 continue
#             img = visualize_s1_path(path)
#             ending = path.split("/")[-1].split('.')[0].split('_')[-1]
#             titles.append(f"S1: {date_str}\n{nodata_perc:.1f}% Nodata")
#             imgs_to_show.append(img)
#             continue
#         print(img_orig.min(), img_orig.max(), img_orig.mean())
#         imgs_to_show.append(img_orig)

#     if len(imgs_to_show) > 0:
#         rows = (len(imgs_to_show) // 5) + 1
#         cols =  (len(imgs_to_show) // rows) + 1
#         f, ax = plt.subplots(rows, max(2, cols), figsize=(30, 7 * rows))
#         ax = ax.flatten()
#         for j in range(len(imgs_to_show)):
#             if "S1" in titles[j]:
#                 ax[j].imshow(imgs_to_show[j])
#             else:
#                 ax[j].imshow(imgs_to_show[j], vmin=-2000, vmax=6000, cmap=red_to_green)
#             ax[j].set_title(titles[j], fontsize=15)

#         mask = rasterio.open(mask_path).read(1)
#         date = mask_path.split("/")[-1].split('_')[1]
#         date_str = f"{date[:4]}-{date[4:6]}-{date[6:8]}"
#         if mask.shape[0] != img.shape[1]:
#             import cv2
#             mask = cv2.resize(mask, (img.shape[1], img.shape[0]), interpolation=cv2.INTER_NEAREST)
#         ax[j+1].imshow(img)
#         # plt.imshow(np.ma.masked_where(mask != 0, mask), cmap="winter", alpha=0.5)
#         ax[j+1].imshow(np.ma.masked_where(mask != 255, mask), cmap="cool" if "_MUX_" in path else "autumn", alpha=0.5)
#         ax[j+1].set_title(f"{date_str}", fontsize=15)
#         # Hide empty axes
#         for ax_idx in range(len(ax)):
#             if is_ax_empty(ax[ax_idx]):  # type: ignore
#                 ax[ax_idx].set_visible(False)  # type: ignore
#         plt.tight_layout()
#         plt.show()

In [ ]:
for mask_path in glob.glob(f"data/*/*_mask.tif"):
    mask = tifffile.imread(mask_path)
    print(mask_path, np.unique(mask))

In [ ]:
np.unique(mask)

In [ ]:
plt.imshow(mask)

In [ ]:
bands = [
    "AOT",
    "B01",
    "B02",
    "B03",
    "B04",
    "B05",
    "B06",
    "B07",
    "B08",
    "B09",
    "B11",
    "B12",
    "B8A",
    "SCL",
    "WVP"
]
bands.index("B12"), bands.index("B08"), bands.index("B04")

In [ ]:
counts = {}
for count_type in ["s2_l2a", "s2_l2a_ndvi", "s2_l2a_gndvi", "s2_l2a_ndwi", "s2_16d", "s2_16d_ndvi", "s2_16d_gndvi", "s2_16d_ndwi"]:
    counts[count_type] = 0
no_s1 = 0
for event_id in event_ids:
    print(f"{event_id}")
    s2_paths = glob.glob(f"data/{event_id}/sentinel2/*")
    tif_paths = glob.glob(f"data/{event_id}/sentinel2/*.tif")
    if len([k for k in tif_paths if "_MSIL2A_" in k]) > 0:
        counts["s2_l2a"] += 1
    # not "ndvi" in k and not "ndwi" in k
    print(f"   S2: {len(s2_paths):2} paths")
    s1_paths = glob.glob(f"data/{event_id}/sentinel1/*.tif")
    if len(s1_paths) == 0:
        no_s1 += 1
    print(f"   S1: {len(s1_paths):2} paths")
    # print(s1_paths)
    # ls_paths = glob.glob(f"data/{event_id}/landsat/*")
    # print(f"   LS: {len(ls_paths):2} paths")
    # cb_paths = glob.glob(f"data/{event_id}/cbers4a/*")
    # print(f"   CB: {len(cb_paths):2} paths")

In [ ]:
counts["s2_l2a"]

In [ ]:
import os
import re
from pathlib import Path
from collections import defaultdict, Counter
from typing import Iterable, Union, Tuple

# ---------------------- Config ----------------------
BASE_DIR = Path("data")  # change if you want to scan the filesystem
SENSORS = ["sentinel2", "sentinel1", "landsat", "cbers4a"]

# Indices to detect in filenames (lowercase match)
INDEX_KEYS = ["ndvi", "gndvi", "ndwi", "nbr", "rvi", "rfdi", "nrpb", "vv_vh_ratio"]

# Optional: pretty labels for combinations
SENSOR_ALIASES = {
    "sentinel1": "s1",
    "sentinel2": "s2",
    "landsat": "ls",
    "cbers4a": "cbers",
}

# ------------------- Helpers ------------------------
def is_raster(path: Union[str, Path]) -> bool:
    p = str(path).lower()
    return p.endswith(".tif") and not p.endswith(".tif.aux.xml")

def get_event_id(path: Union[str, Path]) -> str:
    parts = Path(path).parts
    try:
        i = parts.index("data")
        return parts[i + 1]
    except (ValueError, IndexError):
        m = re.search(r"data[/\\]([^/\\]+)[/\\]", str(path))
        return m.group(1) if m else "UNKNOWN"

def get_sensor(path: Union[str, Path]) -> str:
    parts = Path(path).parts
    try:
        i = parts.index("data")
        return parts[i + 2]
    except (ValueError, IndexError):
        m = re.search(r"data[/\\][^/\\]+[/\\]([^/\\]+)[/\\]", str(path))
        return m.group(1) if m else "UNKNOWN"

def detect_index(filename: str) -> Union[str, None]:
    base = os.path.basename(filename).lower()
    for key in INDEX_KEYS:
        if re.search(rf"(^|[^a-z]){re.escape(key)}($|[^a-z])", base):
            return key
    return None

def detect_raw_type(sensor: str, filename: str) -> Union[str, None]:
    """
    Heuristics for "raw" product-type buckets per sensor family.
    Extend/adjust to your naming conventions if needed.
    """
    base = os.path.basename(filename)

    if sensor == "sentinel2":
        if "S2-16D" in base:
            return "16D"
        if "MSIL2A" in base:
            return "L2A"
        return None

    if sensor == "landsat":
        if "LANDSAT-16D" in base:
            return "16D"
        if re.search(r"L[CT]0[0-9]_L2SP", base):  # LC08/LC09 L2SP
            return "L2SP"
        return None

    if sensor == "sentinel1":
        if re.search(r"S1[AB]_IW_GRDH", base):
            return "GRDH"
        return None

    if sensor == "cbers4a":
        if "MUX_CBERS_4A_MUX_RAW" in base:
            return "MUX_RAW"
        if "WPM_CBERS4A_WPM_PCA" in base:
            return "WPM_PCA"
        return None

    return None

def iter_paths(root: Union[str, Path]) -> Iterable[Path]:
    root = Path(root)
    if root.is_dir():
        yield from root.rglob("*")
    else:
        # If 'root' is a file containing paths, adapt here to read lines
        yield root

# ------------------- Core logic ---------------------
def analyze(paths: Iterable[Union[str, Path]]):
    """
    Returns:
      per_sensor_index_counts: {sensor: Counter({index: count, ...}), ...}
      per_sensor_raw_counts:   {sensor: Counter({rawType: count, ...}), ...}
      missing_counts:          Counter({sensor: num_events_with_no_data, ...})
      missing_event_ids:       {sensor: set(event_ids_with_no_data)}
      combo_counts:            Counter({combo_label: num_events})
    """
    per_sensor_index_counts = defaultdict(Counter)
    per_sensor_raw_counts = defaultdict(Counter)

    # Track whether each (event_id, sensor) has ANY raster at all
    has_data = defaultdict(lambda: {s: False for s in SENSORS})
    all_event_ids = set()

    for p in paths:
        p = Path(p)
        if not p.is_file():
            continue

        sensor = get_sensor(p)
        event_id = get_event_id(p)
        all_event_ids.add(event_id)

        if sensor not in SENSORS:
            continue

        if is_raster(p):
            # presence flag
            has_data[event_id][sensor] = True

            # indices per sensor
            idx = detect_index(p.name)
            if idx:
                per_sensor_index_counts[sensor][idx] += 1

            # raw/product type per sensor
            raw_type = detect_raw_type(sensor, p.name)
            if raw_type:
                per_sensor_raw_counts[sensor][raw_type] += 1

    # Compute missing events per sensor
    missing_event_ids = {s: set() for s in SENSORS}
    for ev in all_event_ids:
        row = has_data.get(ev, {s: False for s in SENSORS})
        for s in SENSORS:
            if not row.get(s, False):
                missing_event_ids[s].add(ev)

    missing_counts = Counter({s: len(missing_event_ids[s]) for s in SENSORS})

    # Build combination labels per event
    combo_counts = Counter()
    for ev in all_event_ids:
        present = [s for s in SENSORS if has_data[ev].get(s, False)]
        if len(present) == 0:
            combo_label = "none"
        elif len(present) == 1:
            # "only s2", etc.
            pl = SENSOR_ALIASES.get(present[0], present[0])
            combo_label = f"only {pl}"
        else:
            # "s1+s2", "s2+ls", etc. (use aliases, keep SENSORS order)
            parts = [SENSOR_ALIASES.get(s, s) for s in present]
            combo_label = "+".join(parts)
        combo_counts[combo_label] += 1

    return (per_sensor_index_counts,
            per_sensor_raw_counts,
            missing_counts,
            missing_event_ids,
            combo_counts)

# ------------------- Pretty printing ----------------
def print_summary(per_sensor_index_counts,
                  per_sensor_raw_counts,
                  missing_counts,
                  missing_event_ids,
                  combo_counts,
                  show_missing_lists=False):
    print("\n=== INDEX COUNTS PER SENSOR ===")
    for sensor in SENSORS:
        cnt = per_sensor_index_counts.get(sensor, Counter())
        print(f"\n[{sensor}]")
        if cnt:
            for k, v in cnt.most_common():
                print(f"  {k:12s} {v:6d}")
        else:
            print("  (no index rasters found)")

    print("\n=== RAW/PRODUCT TYPE COUNTS PER SENSOR ===")
    for sensor in SENSORS:
        cnt = per_sensor_raw_counts.get(sensor, Counter())
        print(f"\n[{sensor}]")
        if cnt:
            for k, v in cnt.most_common():
                print(f"  {k:12s} {v:6d}")
        else:
            print("  (no raw/product-type rasters found)")

    print("\n=== EVENTS WITH NO DATA BY SENSOR ===")
    for sensor in SENSORS:
        print(f"  {sensor:10s}: {missing_counts[sensor]:6d} events with no rasters")

    print("\n=== SENSOR PRESENCE COMBINATIONS (per event) ===")
    for label, n in combo_counts.most_common():
        print(f"  {label:15s} {n:6d}")

    if show_missing_lists:
        print("\n=== LIST OF EVENT IDS WITH NO DATA (per sensor) ===")
        for sensor in SENSORS:
            ids = sorted(missing_event_ids[sensor])
            print(f"\n[{sensor}] ({len(ids)}):")
            if ids:
                print(", ".join(ids))
            else:
                print("  (none)")

# ------------------- Example usage ------------------
if __name__ == "__main__":
    # Option A: Walk your on-disk tree under ./data
    results = analyze(iter_paths(BASE_DIR))
    print_summary(*results, show_missing_lists=False)

    # Option B: Use prebuilt lists instead of walking the FS (uncomment):
    # sample_all = s2_list + s1_list + landsat_list + cbers_list
    # results = analyze(sample_all)
    # print_summary(*results, show_missing_lists=True)


In [ ]:
events_with = sorted([k for k in glob.glob("data/*/sentinel2/*.tif") if not "ndvi" in k and not "ndwi" in k])
len(s2_band_paths)

In [ ]:
s2_band_paths = sorted([k for k in glob.glob("data/*/sentinel2/*.tif") if not "ndvi" in k and not "ndwi" in k])
len(s2_band_paths)

In [ ]:
len(np.unique([k.split("/")[1] for k in s2_band_paths if "_S2-16D_V2_" in k]))

In [ ]:
len([k for k in s2_band_paths if "_S2-16D_V2_" in k])

In [ ]:
len(np.unique([k.split("/")[1] for k in s2_band_paths if "L2A" in k]))

In [ ]:
len([k for k in s2_band_paths if "L2A" in k])

In [ ]:
s2_band_paths = sorted([k for k in glob.glob("data/*/sentinel2/*.tif") if "_ndvi" in k])
len(s2_band_paths)

In [ ]:
img_orig = rasterio.open(s2_band_paths[rand_int]).read()
img_orig.shape

In [ ]:
img_orig.min()

In [ ]:
img_orig.max()

In [ ]:
from matplotlib.colors import LinearSegmentedColormap
red_to_green = LinearSegmentedColormap.from_list("RedGreen", ["red", "green"])

for k in range(10):
    rand_int = np.random.randint(len(s2_band_paths))
    ndvi = rasterio.open(s2_band_paths[rand_int]).read(1)
    print(ndvi.min(), ndvi.max())
    # img_orig = tifffile.imread(s2_band_paths[rand_int])
    # img = visualize_s2_concat_bands_path(s2_band_paths[rand_int])
    ending = s2_band_paths[rand_int].split("/")[-1]

    f, ax = plt.subplots(1, 2, figsize=(30, 10))
    ax[0].imshow(ndvi, vmin=-2000, vmax=6000, cmap=red_to_green)
    ax[0].set_title(f"{ending[:30]}", fontsize=15)
    # ax[0].set_title(f"{ending[:30]}: {img_orig.shape}", fontsize=15)
    # pred = ((preds[i].astype(np.float32) / 255.0) > threshold) * 1
    ax[1].imshow(ndvi, vmin=-2000, vmax=6000, cmap=red_to_green)
    # axarr[1].imshow(np.ma.masked_where(pred == 0, pred), cmap="spring", alpha=alpha)
    # title_string = f"FPs: {fps[i]:.0f} = {100 * fps[i] / fp_sum:.1f}%, FNs: {fns[i]:.0f} = {100 * fns[i] / fn_sum:.1f}%, TPs: {tps[i]:.0f}"
    # axarr[1].set_title(f"(Pred) {title_string}", fontsize=fontsize)
    
    # label = tifffile.imread(GT_paths[i])
    # nan_mask = label == 255
    # label = ((label >= 2) & (label <= 4)).astype(np.uint8)
    # label[nan_mask] = 0
    # ax[1].imshow(img)
    # axarr[2].imshow(np.ma.masked_where(label != 1, label), cmap="spring", alpha=alpha)
    ax[1].set_title(f"image with GT mask", fontsize=15)
    
    plt.tight_layout()
    plt.show()